# Unique pathogen target between human and human-virus


In [2]:
from Bio import SeqIO
import gzip
from collections import defaultdict

# Function to read and preprocess FASTA sequences
def read_fasta(file_path, min_length=30):
    """
    Reads a FASTA file, extracts unique sequences, filters out ambiguous residues,
    applies length filtering, and stores sequences along with their headers.
    
    :param file_path: Path to the FASTA file.
    :param min_length: Minimum allowed sequence length.
    :return: Dictionary {sequence: [headers]}.
    """
    unique_sequences = defaultdict(list)  # Dictionary to store sequences and their headers

    # Open file normally or as a gzip file
    if file_path.endswith(".gz"):
        handle = gzip.open(file_path, "rt")  # Open compressed file in text mode
    else:
        handle = open(file_path, "r")  # Open normal FASTA file

    with handle:
        for record in SeqIO.parse(handle, "fasta"):
            seq = str(record.seq).strip().upper()  # Convert to uppercase, remove spaces/newlines
            
            # Apply length filtering and remove sequences with ambiguous residues
            if len(seq) >= min_length and not any(x in seq for x in ["X", "B", "Z", "*", "U"]):
                unique_sequences[seq].append(record.id)  # Store sequence with its header

    return unique_sequences  # Returns {sequence: [headers]}

# Load human and viral protein sequences
human_proteins = read_fasta(r"DATASET\\Human_prion .fasta")
viral_proteins = read_fasta(r"DATASET\\Virus_prion.fasta")

# Convert all human and viral sequences into a single concatenated string
human_protein_string = "".join(human_proteins.keys())
viral_protein_string = "".join(viral_proteins.keys())

# Compute total length of concatenated sequences
total_length = len(human_protein_string) + len(viral_protein_string)

# Print summary of preprocessing
print(f"Total length of concatenated sequences: {total_length}")
print(f"Unique human protein sequences: {len(human_proteins)}")
print(f"Unique viral protein sequences: {len(viral_proteins)}")
print(f"Total length of human protein string: {len(human_protein_string)}")
print(f"Total length of viral protein string: {len(viral_protein_string)}")

# Function to return concatenated sequences for further processing (e.g., sequence alignment)
def get_protein_strings():
    """
    Returns the concatenated protein sequences for further processing.
    
    :return: (human_protein_string, viral_protein_string)
    """
    return human_protein_string, viral_protein_string

Total length of concatenated sequences: 1854
Unique human protein sequences: 1
Unique viral protein sequences: 1
Total length of human protein string: 581
Total length of viral protein string: 1273


In [3]:
# Print the first 100 characters of human_protein_string
print("\nFirst 100 characters of human protein sequence:")
print(human_protein_string[:100])  

# Print the next 100 characters of viral_protein_string
print("\nNext 100 characters of viral protein sequence:")
print(viral_protein_string[:100])  


First 100 characters of human protein sequence:
MVAEVCSMPAASAVKKPFDLRSKMGKWCHHRFPCCRGSGKSNMGTSGDHDDSFMKTLRSKMGKCCHHCFPCCRGSGTSNVGTSGDHDNSFMKTLRSKMGK

Next 100 characters of viral protein sequence:
MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNI


In [5]:
import numpy as np

def read_fasta(file_path):
    """
    Reads sequences from a FASTA file and returns a list of sequences.
    """
    sequences = []
    with open(file_path, "r") as file:
        seq = ""
        for line in file:
            if line.startswith(">"):
                if seq:
                    sequences.append(seq)
                seq = ""  # Reset for new sequence
            else:
                seq += line.strip()
        if seq:  # Append the last sequence
            sequences.append(seq)
    return sequences

def nw_last_row(seq1, seq2, gap_penalty=-2, match_score=1, mismatch_score=-1):
    """
    Computes only the last row of the Needleman-Wunsch DP table to save space.
    """
    prev_row = np.zeros(len(seq2) + 1, dtype=int)
    
    for j in range(len(seq2) + 1):
        prev_row[j] = j * gap_penalty
    
    for i in range(1, len(seq1) + 1):
        curr_row = np.zeros(len(seq2) + 1, dtype=int)
        curr_row[0] = i * gap_penalty
        
        for j in range(1, len(seq2) + 1):
            match = prev_row[j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score)
            delete = prev_row[j] + gap_penalty
            insert = curr_row[j - 1] + gap_penalty
            curr_row[j] = max(match, delete, insert)
        
        prev_row = curr_row  # Move to the next row
    
    return prev_row

def needleman_wunsch(seq1, seq2, gap_penalty=-2, match_score=1, mismatch_score=-1):
    """
    Needleman-Wunsch algorithm for base case alignment when sequences are very short.
    """
    n, m = len(seq1), len(seq2)
    dp = np.zeros((n + 1, m + 1), dtype=int)
    
    # Initialize the DP table
    for i in range(n + 1):
        dp[i][0] = i * gap_penalty
    for j in range(m + 1):
        dp[0][j] = j * gap_penalty
    
    # Fill the DP table
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match = dp[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score)
            delete = dp[i - 1][j] + gap_penalty
            insert = dp[i][j - 1] + gap_penalty
            dp[i][j] = max(match, delete, insert)
    
    # Traceback to get the aligned sequences
    align1, align2 = "", ""
    i, j = n, m
    while i > 0 or j > 0:
        if i > 0 and j > 0 and dp[i][j] == dp[i - 1][j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_score):
            align1 = seq1[i - 1] + align1
            align2 = seq2[j - 1] + align2
            i -= 1
            j -= 1
        elif i > 0 and dp[i][j] == dp[i - 1][j] + gap_penalty:
            align1 = seq1[i - 1] + align1
            align2 = "-" + align2
            i -= 1
        else:
            align1 = "-" + align1
            align2 = seq2[j - 1] + align2
            j -= 1
    
    return align1, align2

def hirschberg(seq1, seq2, gap_penalty=-2, match_score=1, mismatch_score=-1):
    """
    Hirschberg's Algorithm for global sequence alignment (O(n) space complexity).
    """
    if len(seq1) == 0:
        return "-" * len(seq2), seq2
    if len(seq2) == 0:
        return seq1, "-" * len(seq1)
    if len(seq1) == 1 or len(seq2) == 1:
        return needleman_wunsch(seq1, seq2, gap_penalty, match_score, mismatch_score)
    
    mid = len(seq1) // 2
    left_score = nw_last_row(seq1[:mid], seq2, gap_penalty, match_score, mismatch_score)
    right_score = nw_last_row(seq1[mid:][::-1], seq2[::-1], gap_penalty, match_score, mismatch_score)[::-1]
    
    # Fix: Handle tie cases explicitly
    split_scores = left_score + right_score
    split_idx = np.where(split_scores == np.max(split_scores))[0][0]  # Select the first max value

    # Recursively align both halves
    left_alignment = hirschberg(seq1[:mid], seq2[:split_idx], gap_penalty, match_score, mismatch_score)
    right_alignment = hirschberg(seq1[mid:], seq2[split_idx:], gap_penalty, match_score, mismatch_score)
    
    return left_alignment[0] + right_alignment[0], left_alignment[1] + right_alignment[1]

# Compute total unique sequences
total_unique_rows = len(human_protein_string) + len(viral_protein_string)

# Print dataset info
print(f"Total unique data points (rows) after filtering: {total_unique_rows}")
print(f"Total unique human proteins: {len(human_protein_string)}")
print(f"Total unique viral proteins: {len(viral_protein_string)}")

# Perform Hirschberg alignment on full protein sequences
align1, align2 = hirschberg(human_protein_string, viral_protein_string)
print("\nHirschberg Algorithm Alignment:")
print("A1:", align1)
print("A2:", align2)

Total unique data points (rows) after filtering: 1854
Total unique human proteins: 581
Total unique viral proteins: 1273

Hirschberg Algorithm Alignment:
A1: M-VAEVC----SM-----------PAA---SA---VKKP---FD---LRSKMGK---------WCH--H--------RF--PCCR---G----SG-KSNM--GTS-GDHDDSFMKTLRSKMGKCCHH---C-FPCCR----G--------SGT-SN--VGTSGDHDN----S--FMKTLRSKMG--KWCCHC-FPCCRGSGKSNVG-TWGDYD-DSA--FM--EPRYHVR------RED-LDKLHRAAW--------W--GK----V----PRKDLIVMLRD-T--DMNKR--DKQKRTALHLASAN---GN---SEV-VQLLLDR-RCQ----L-------N------VLD-NKKRTALIKA---VQCQEDE-----C--VLM--L--LEHG---ADGN-IQ-DEY-----GNT---ALH-YA---------IY-NEDKLMAKA------L--L-----LYGA--DI--E-----SK--NKC-GLT---PLLL-G------VHEQKQQVVK--F-LI---------KK-----KA---N-----L---------NA--LDRY--GR----T--A------L-ILAVC-C--GSAS-IV---NLL-----LEQ--N---V------D-------V-S--S---QD----L--------S-------G-------QT-------ARE--------Y-----A---V--S--SHHHV----IC---ELL--SDYKEKQMLK--IS--S-ENSNPE-QD------LK--LTSE--E------ES--QRL---KVSE-------N-SQ--PE--KMSQEPEI-----NK----D---------C--

In [6]:
def extract_unique_non_conserved_regions(aligned_human, aligned_virus):
    """
    Extracts unique viral regions from the aligned sequences by:
    1. Identifying positions where the human protein has gaps ("-").
    2. Removing viral residues that match human residues (conserved regions).
    
    :param aligned_human: Aligned human protein sequence (A1)
    :param aligned_virus: Aligned viral protein sequence (A2)
    :return: Unique viral sequence without human similarity
    """
    unique_viral_seq = ""

    for i in range(len(aligned_human)):
        # Keep viral sequence only if:
        # - The human sequence has a gap (viral-specific region)
        # - The viral residue is NOT the same as the human residue (not conserved)
        if aligned_human[i] == "-" and aligned_virus[i] != "-":
            unique_viral_seq += aligned_virus[i]

    return unique_viral_seq

# Extract unique viral sequence
unique_viral_sequence = extract_unique_non_conserved_regions(align1, align2)

print("\nUnique Viral Sequence (Safe for Drug Targeting):")
print(unique_viral_sequence)


Unique Viral Sequence (Safe for Drug Targeting):
FLPLVQCVNLTTRTQLYTNTRGDKVSSVFLPFFSNVTAIVSGTNGTKDNFNDVYFAEIRFIKVEDPFLVYYHKNNKEFRFEYVQPNFVHRQGALIGINITTTPGDSSSGTAAAYYGYLQGITALVEKYQTRVNITNCPFGEVFATRFASWDYSFSTFKYGTKNDNVYVGRQIAPGKINLPDDFTGCVWGGNYNYYRFRKSNERSTIYQAGPCECYFYFQPTNGLSEHAPATVCGPSTNLVKCVFNFNGTGTGVLTESKFQFDIADTDVRDPQTEPSFVPGTNQVAVDVCTEPVAIHAQLTPTWRYTGNVFRAGCIGAEHVNNYECDIPIAGICASYQTNSPRRVASQSIIATMSLGENSAYNNTNFTVTTPVMYGDTLGSFCTQRAAVQDKNTQFAQIYIKDFGGFFILPSEDLLFVTLAAGFIKQYGDLGVMIQYTLLAITSFAGAAQFAMGIGVTQNVLYKLIANSAIGKIQSLSSQVNQALNVSSVLNDLLKVAEVQDRILLTQQLIRAIRASANLAATMSLGQRVDFCGKGYHMPQAPFLHVTYVPAQKNFTTAPAICHAHFPVFVTHWFVTQFYPQNTFVSGNCDVVIGIVNNTVYDPQPSFKLDKTSPDVDISGIASVVNIQEDRNVAKNLNSLDLQYQYIWPWYIWGFIAGMVTIMCMTSCCSCLKGCCSCGSCCKFDDDSVGVY


In [7]:
def similarity_score(seq1, seq2):
    """
    Computes the similarity score between two aligned sequences and returns a percentage.
    
    Parameters:
    - seq1 (str): First aligned sequence (e.g., human protein).
    - seq2 (str): Second aligned sequence (e.0.g., viral protein).
    
    Returns:
    - float: Similarity score out of 100.
    """
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must be aligned and of the same length.")

    match_score = 1  # Points for a match
    total_length = len(seq1)  # Length of aligned sequences
    match_count = sum(1 for a, b in zip(seq1, seq2) if a == b and a != "-")

    similarity_percentage = (match_count / total_length) * 100
    return round(similarity_percentage, 2)

# Example Usage
seq1 = align1
seq2 = align2
score = similarity_score(seq1, seq2)
print(f"Similarity Score: {score}%")

Similarity Score: 21.45%


In [9]:
import matplotlib.pyplot as plt
from Bio import PDB

def ramachandran_plot(pdb_file):
    parser = PDB.PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)
    phi_psi_angles = []

    for model in structure:
        for chain in model:
            polypeptides = PDB.PPBuilder().build_peptides(chain)
            for poly_index, poly in enumerate(polypeptides):
                phi_psi_angles.extend(poly.get_phi_psi_list())

    # Extract valid angles
    phi_angles = [angle[0] for angle in phi_psi_angles if angle[0] is not None]
    psi_angles = [angle[1] for angle in phi_psi_angles if angle[1] is not None]

    # Plot Ramachandran Plot
    plt.figure(figsize=(7, 7))
    plt.xlim(-180, 180)
    plt.ylim(-180, 180)
    plt.xlabel("Phi (ϕ) angles")
    plt.ylabel("Psi (ψ) angles")
    plt.title("Ramachandran Plot")
    plt.scatter(phi_angles, psi_angles, s=10, color="blue", alpha=0.5)
    plt.axhline(0, color="black", linewidth=0.5)
    plt.axvline(0, color="black", linewidth=0.5)
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.show()

# ramachandran_plot("C:\\Users\\vikas\\Downloads\\model1 (1).pdb") 


In [12]:
values = [7.5, 7.4, 7.3, 7.1, 7.0, 7.0, 7.0, 7.0]  # affinity energy 
weights = [0.0, 1.27, 2.373, 4.301, 4.072, 3.689, 4.108, 4.214]  # rmsd/lb
max_weight = 10.0  # Seting a cutoff for total rmsd/lb

In [13]:
def knapsack(values, weights, max_weight):
    n = len(values)
    W = int(max_weight * 100)  # scale to integers
    wt = [int(w * 100) for w in weights]
    
    dp = [[0] * (W + 1) for _ in range(n + 1)]
    
    for i in range(1, n + 1):
        for w in range(W + 1):
            if wt[i - 1] <= w:
                dp[i][w] = max(dp[i - 1][w], dp[i - 1][w - wt[i - 1]] + values[i - 1])
            else:
                dp[i][w] = dp[i - 1][w]
    
    # Backtrack to find selected ligands
    res = dp[n][W]
    w = W
    selected = []
    
    for i in range(n, 0, -1):
        if res <= 0:
            break
        if res == dp[i - 1][w]:
            continue
        else:
            selected.append(i - 1)
            res -= values[i - 1]
            w -= wt[i - 1]
    
    return selected


In [14]:
selected = knapsack(values, weights, max_weight)

print("Selected Ligands:")
for i in selected[::-1]:  # Reverse for original order
    print(f"Rank {i+1}: Energy = {-values[i]} kcal/mol, rmsd/lb = {weights[i]} Å")

Selected Ligands:
Rank 1: Energy = -7.5 kcal/mol, rmsd/lb = 0.0 Å
Rank 2: Energy = -7.4 kcal/mol, rmsd/lb = 1.27 Å
Rank 3: Energy = -7.3 kcal/mol, rmsd/lb = 2.373 Å
Rank 4: Energy = -7.1 kcal/mol, rmsd/lb = 4.301 Å


In [15]:
def parse_pocinfo(file_path):
    """
    Parse the .pocInfo file and return the pocket with the highest Vol_ms.
    Returns a dictionary with keys 'id' and 'volume'.
    
    Expected file format example (tab-separated):
    
    POC:	Molecule	ID	N_mth	Area_sa	Area_ms	Vol_sa	Vol_ms	Lenth	cnr
    POC:	j_67f7907b2f292	1	83	46455.182	51202.342	191294.408	258435.017	28589.138	10852
    ...
    """
    best_pocket = None
    max_volume = -1

    with open(file_path, 'r') as f:
        for line in f:
            # Skip header line if present
            if line.startswith("POC:") and "Molecule" in line:
                continue
            
            if line.startswith("POC:"):
                parts = line.strip().split('\t')
                if len(parts) >= 7:
                    try:
                        # parts[2] is the pocket ID (as string) and parts[6] is Vol_sa, parts[7] is Vol_ms
                        pocket_id = int(parts[2])
                        vol_ms = float(parts[7])  # Use Vol_ms (index 7)
                        if vol_ms > max_volume:
                            max_volume = vol_ms
                            best_pocket = {'id': pocket_id, 'volume': vol_ms}
                    except ValueError:
                        continue

    if best_pocket:
        print(f"Best Pocket ID: {best_pocket['id']} with Vol_ms = {best_pocket['volume']}")
    else:
        print("No valid pocket data found.")
    
    return best_pocket


def extract_poc_coordinates(file_path, target_pocket_id):
    """
    Extract the coordinates (x, y, z) from the .poc file for the target pocket.
    
    For this sample .poc file, we assume that ATOM records belong to a single pocket 
    (target_pocket_id). If the file contains records for multiple pockets, additional 
    logic is needed to separate them.
    
    Expected .poc file sample (each ATOM record is space-delimited):
    ATOM      1  N   GLN A   6     164.246 117.137 190.085  1.00  0.00   1  POC
    ...
    """
    coords = []
    
    with open(file_path, 'r') as f:
        for line in f:
            # Process only lines that start with "ATOM"
            if line.startswith("ATOM"):
                parts = line.strip().split()
                if len(parts) >= 9:
                    try:
                        # According to the sample:
                        # parts[6] = x, parts[7] = y, parts[8] = z
                        x, y, z = map(float, (parts[6], parts[7], parts[8]))
                        coords.append((x, y, z))
                    except ValueError:
                        continue

    if not coords:
        print(f"No coordinates found for Pocket {target_pocket_id}.")
        return None

    # Calculate the centroid (average coordinates) of the pocket
    x_avg = sum(coord[0] for coord in coords) / len(coords)
    y_avg = sum(coord[1] for coord in coords) / len(coords)
    z_avg = sum(coord[2] for coord in coords) / len(coords)
    
    print(f"Center of Pocket {target_pocket_id}: ({x_avg:.3f}, {y_avg:.3f}, {z_avg:.3f})")
    return (x_avg, y_avg, z_avg)


# === Replace these paths with your actual file locations ===
pocinfo_path = "j_67f7907b2f292.pocInfo"
poc_path = "j_67f7907b2f292.poc"

# === Run the pocket selection and coordinate extraction ===
best_pocket = parse_pocinfo(pocinfo_path)

if best_pocket is not None:
    # For this example, we assume all ATOM records in the .poc file belong to pocket ID 1.
    coordinates = extract_poc_coordinates(poc_path, best_pocket['id'])

    if coordinates is not None:
        with open("best_pocket_coordinates.txt", "w") as f:
            f.write(f"{coordinates[0]:.3f}, {coordinates[1]:.3f}, {coordinates[2]:.3f}\n")
        print("Coordinates saved to best_pocket_coordinates.txt")


Best Pocket ID: 1 with Vol_ms = 258435.017
Center of Pocket 1: (162.197, 150.962, 157.402)
Coordinates saved to best_pocket_coordinates.txt
